In [1]:
import json
import pandas as pd
import math
from tqdm import tqdm 
import cv2
import numpy as np
import os

import torch 
import torchvision
import torch.nn as nn 
from IPython.display import Image 
from torchvision import transforms
import matplotlib.pyplot as plt
import random
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
seed = 12345
random.seed(seed)
torch.manual_seed(seed)
import PIL
import requests
  
from sklearn import decomposition    
from sklearn.preprocessing import MinMaxScaler

# Load wrong predictions

In [67]:
def load_preds():
    '''
    load wrong predictions on real data
    '''
    coco_dir = "/content/drive/MyDrive/111 Rendered.ai/RarePlanes/output/coco_instances_results.json"
    images_dir = "/content/drive/MyDrive/111 Rendered.ai/RarePlanes/datasets/synthetic/test/images/"
    test_json_dir = coco_dir
    with open(test_json_dir) as json_file:
        preds_coco = json.load(json_file)
    preds_coco = pd.DataFrame(preds_coco)
    print("There are {} predicted bbox.".format(preds_coco.shape[0]))
    return preds_coco


def get_image_ann():
    '''
    load image annotations
    '''
    coco_dir = "/content/drive/MyDrive/111 Rendered.ai/RarePlanes/datasets/coco_data/aircraft_real_test_coco.json"
    images_dir = "/content/drive/MyDrive/111 Rendered.ai/RarePlanes/datasets/real/test/RarePlanes_test_PS-RGB_tiled.tar.gz (Unzipped Files)/PS-RGB_tiled"
    test_json_dir = coco_dir
    with open(test_json_dir) as json_file:
        test_coco = json.load(json_file)
    img_list = pd.DataFrame(test_coco['images'])
    annotations = pd.DataFrame(test_coco['annotations'])
    ann = annotations.merge(img_list, how='left', left_on="image_id", right_on="id")
    return ann


def get_wrong_pred_img_id(threshold = 0.8):
    '''
    get hesitate predicted img ids
    '''
    img_ids = preds_coco[preds_coco["score"]<threshold].image_id.drop_duplicates().reset_index().image_id
    print("There are {} images with with a bit hesitation (prob<0.8). Let's study those images!".format(img_ids.shape[0]))
    return img_ids


def id2filename(img_ids, ann):
    # get img file dir
    return pd.DataFrame(img_ids, columns=["image_id"]).merge(ann[["image_id","file_name"]], 
                                                             how="left", on="image_id").drop_duplicates()


def load_wrong_images(images_dir = "/content/drive/MyDrive/111 Rendered.ai/RarePlanes/datasets/real/test/RarePlanes_test_PS-RGB_tiled.tar.gz (Unzipped Files)/PS-RGB_tiled/"):
    img_collections = []
    for f in tqdm(img_dirs.file_name):
        im = cv2.imread(images_dir+f)
        img_collections.append(im)
    img_collections = np.asarray(img_collections)  
    print("img_collections shape:",img_collections.shape)
    return img_collections

ann = get_image_ann()
preds_coco = load_preds()
img_ids = get_wrong_pred_img_id()
img_dirs = id2filename(img_ids, ann)
img_collections = load_wrong_images()

There are 6552 predicted bbox.
There are 43 images with with a bit hesitation (prob<0.8). Let's study those images!



img_collections shape: (43, 512, 512, 3)


# Localization